# FinBERT Example Notebook

This notebooks shows how to train and use the FinBERT pre-trained language model for financial sentiment analysis.

## Modules 

In [34]:
from pathlib import Path
import sys
sys.path.append('..')
import argparse
import shutil
import os
import logging
from textblob import TextBlob

from pytorch_pretrained_bert.file_utils import PYTORCH_PRETRAINED_BERT_CACHE
from pytorch_pretrained_bert.modeling import BertForSequenceClassification
from pytorch_pretrained_bert.tokenization import BertTokenizer
from pytorch_pretrained_bert.optimization import *

from finbert.finbert import *
import finbert.utils as tools
from pprint import pprint
from sklearn.metrics import classification_report
%load_ext autoreload
%autoreload 2

project_dir = Path.cwd().parent
pd.set_option('max_colwidth', -1)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [35]:
logging.basicConfig(format = '%(asctime)s - %(levelname)s - %(name)s -   %(message)s',
                    datefmt = '%m/%d/%Y %H:%M:%S',
                    level = logging.ERROR)

## Prepare the model

### Setting path variables:
1. `lm_path`: the path for the pre-trained language model (If vanilla Bert is used then no need to set this one).
2. `cl_path`: the path where the classification model is saved.
3. `cl_data_path`: the path of the directory that contains the data files of `train.csv`, `validation.csv`, `test.csv`.
---

In the initialization of `bertmodel`, we can either use the original pre-trained weights from Google by giving `bm = 'bert-base-uncased`, or our further pre-trained language model by `bm = lm_path`


---
All of the configurations with the model is controlled with the `config` variable. 

In [36]:
lm_path = project_dir/'models'/'language_model'/'finbertTRC2'
cl_path = project_dir/'models'/'classifier_model'/'finbert-sentiment'
cl_data_path = project_dir/'data'/'sentiment_data'

###  Configuring training parameters

You can find the explanations of the training parameters in the class docsctrings. 

In [37]:
# Clean the cl_path
try:
    shutil.rmtree(cl_path) 
except:
    pass

bertmodel = BertForSequenceClassification.from_pretrained(lm_path,cache_dir=None, num_labels=3)


config = Config(   data_dir=cl_data_path,
                   bert_model=bertmodel,
                   num_train_epochs=4,
                   model_dir=cl_path,
                   max_seq_length = 48,
                   train_batch_size = 32,
                   learning_rate = 2e-5,
                   output_mode='classification',
                   warm_up_proportion=0.2,
                   local_rank=-1,
                   discriminate=True,
                   gradual_unfreeze=True )

05/16/2020 13:43:37 - INFO - pytorch_pretrained_bert.modeling -   loading archive file /mnt/c/Users/Fazz/source/repos/finBERT/models/language_model/finbertTRC2 from cache at /mnt/c/Users/Fazz/source/repos/finBERT/models/language_model/finbertTRC2
05/16/2020 13:43:37 - INFO - pytorch_pretrained_bert.modeling -   Model config {
  "attention_probs_dropout_prob": 0.1,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "max_position_embeddings": 512,
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "type_vocab_size": 2,
  "vocab_size": 30522
}

05/16/2020 13:43:44 - INFO - pytorch_pretrained_bert.modeling -   Weights of BertForSequenceClassification not initialized from pretrained model: ['classifier.weight', 'classifier.bias']
05/16/2020 13:43:44 - INFO - pytorch_pretrained_bert.modeling -   Weights from pretrained model not used in BertForSequenceClassification: ['cls.predictions.bias', 'cls.pred

`finbert` is our main class that encapsulates all the functionality. The list of class labels should be given in the prepare_model method call with label_list parameter.

In [38]:
finbert = FinBert(config)

In [39]:
finbert.prepare_model(label_list=['positive','negative','neutral'])

05/16/2020 13:43:44 - INFO - finbert.finbert -   device: cpu n_gpu: 0, distributed training: False, 16-bits training: False
05/16/2020 13:43:45 - INFO - pytorch_pretrained_bert.tokenization -   loading vocabulary file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-vocab.txt from cache at /home/fazz/.pytorch_pretrained_bert/26bc1ad6c0ac742e9b52263248f6d0f00068293b33709fae12320c0e35ccfbbb.542ce4285a40d23a559526243235df47c5f75c197f04f37d1a0c124c32c9a084


## Fine-tune the model

In [40]:
# Get the training examples
train_data = finbert.get_data('train')

In [41]:
model = finbert.create_the_model()

### [Optional] Fine-tune only a subset of the model
The variable `freeze` determines the last layer (out of 12) to be freezed. You can skip this part if you want to fine-tune the whole model.

<span style="color:red">Important: </span>
Execute this step if you want a shorter training time in the expense of accuracy.

In [42]:
# This is for fine-tuning a subset of the model.

freeze = 11

for param in model.bert.embeddings.parameters():
    param.requires_grad = False
    
for i in range(freeze):
    for param in model.bert.encoder.layer[i].parameters():
        param.requires_grad = False

### Training

In [43]:
trained_model = finbert.train(train_examples = train_data, model = model)

05/16/2020 13:43:45 - INFO - finbert.utils -   *** Example ***
05/16/2020 13:43:45 - INFO - finbert.utils -   guid: train-1
05/16/2020 13:43:45 - INFO - finbert.utils -   tokens: [CLS] according to gran , the company has no plans to move all production to russia , although that is where the company is growing . [SEP]
05/16/2020 13:43:45 - INFO - finbert.utils -   input_ids: 101 2429 2000 12604 1010 1996 2194 2038 2053 3488 2000 2693 2035 2537 2000 3607 1010 2348 2008 2003 2073 1996 2194 2003 3652 1012 102 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
05/16/2020 13:43:45 - INFO - finbert.utils -   input_mask: 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
05/16/2020 13:43:46 - INFO - finbert.utils -   segment_ids: 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
05/16/2020 13:43:46 - INFO - finbert.utils -   label: neutral (id = 2)
05/16/2020 13:43:46 - INFO - finbert.finbert -   ***** Loading da

05/16/2020 13:45:10 - INFO - finbert.utils -   *** Example ***
05/16/2020 13:45:10 - INFO - finbert.utils -   guid: validation-1
05/16/2020 13:45:10 - INFO - finbert.utils -   tokens: [CLS] another noticeable thing is that the search for tata and air ##tel brands was mostly related to ` broadband connections ' . [SEP]
05/16/2020 13:45:10 - INFO - finbert.utils -   input_ids: 101 2178 17725 2518 2003 2008 1996 3945 2005 23236 1998 2250 9834 9639 2001 3262 3141 2000 1036 19595 7264 1005 1012 102 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
05/16/2020 13:45:10 - INFO - finbert.utils -   input_mask: 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
05/16/2020 13:45:10 - INFO - finbert.utils -   segment_ids: 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
05/16/2020 13:45:10 - INFO - finbert.utils -   label: neutral (id = 2)
05/16/2020 13:45:10 - INFO - finbert.finbert -   ***** Loading data ***


Validation losses: [0.9659676849842072]
No best model found
Epoch:  25%|██▌       | 1/4 [01:33<04:39, 93.18s/it]

05/16/2020 13:47:50 - INFO - finbert.utils -   *** Example ***
05/16/2020 13:47:50 - INFO - finbert.utils -   guid: validation-1
05/16/2020 13:47:50 - INFO - finbert.utils -   tokens: [CLS] another noticeable thing is that the search for tata and air ##tel brands was mostly related to ` broadband connections ' . [SEP]
05/16/2020 13:47:50 - INFO - finbert.utils -   input_ids: 101 2178 17725 2518 2003 2008 1996 3945 2005 23236 1998 2250 9834 9639 2001 3262 3141 2000 1036 19595 7264 1005 1012 102 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
05/16/2020 13:47:50 - INFO - finbert.utils -   input_mask: 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
05/16/2020 13:47:50 - INFO - finbert.utils -   segment_ids: 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
05/16/2020 13:47:50 - INFO - finbert.utils -   label: neutral (id = 2)
05/16/2020 13:47:50 - INFO - finbert.finbert -   ***** Loading data ***


Validation losses: [0.9659676849842072, 0.7577664852142334]
Epoch:  50%|█████     | 2/4 [04:13<03:46, 113.30s/it]

05/16/2020 13:51:24 - INFO - finbert.utils -   *** Example ***
05/16/2020 13:51:24 - INFO - finbert.utils -   guid: validation-1
05/16/2020 13:51:24 - INFO - finbert.utils -   tokens: [CLS] another noticeable thing is that the search for tata and air ##tel brands was mostly related to ` broadband connections ' . [SEP]
05/16/2020 13:51:24 - INFO - finbert.utils -   input_ids: 101 2178 17725 2518 2003 2008 1996 3945 2005 23236 1998 2250 9834 9639 2001 3262 3141 2000 1036 19595 7264 1005 1012 102 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
05/16/2020 13:51:24 - INFO - finbert.utils -   input_mask: 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
05/16/2020 13:51:24 - INFO - finbert.utils -   segment_ids: 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
05/16/2020 13:51:24 - INFO - finbert.utils -   label: neutral (id = 2)
05/16/2020 13:51:24 - INFO - finbert.finbert -   ***** Loading data ***


Validation losses: [0.9659676849842072, 0.7577664852142334, 0.5364635288715363]
Epoch:  75%|███████▌  | 3/4 [07:47<02:23, 143.47s/it]

05/16/2020 13:54:32 - WARNING - pytorch_pretrained_bert.optimization -   Training beyond specified 't_total'. Learning rate multiplier set to 0.0. Please set 't_total' of WarmupLinearSchedule correctly.
05/16/2020 13:54:53 - WARNING - pytorch_pretrained_bert.optimization -   Training beyond specified 't_total'. Learning rate multiplier set to 0.0. Please set 't_total' of WarmupLinearSchedule correctly.
05/16/2020 13:55:12 - WARNING - pytorch_pretrained_bert.optimization -   Training beyond specified 't_total'. Learning rate multiplier set to 0.0. Please set 't_total' of WarmupLinearSchedule correctly.
05/16/2020 13:55:15 - INFO - finbert.utils -   *** Example ***
05/16/2020 13:55:15 - INFO - finbert.utils -   guid: validation-1
05/16/2020 13:55:15 - INFO - finbert.utils -   tokens: [CLS] another noticeable thing is that the search for tata and air ##tel brands was mostly related to ` broadband connections ' . [SEP]
05/16/2020 13:55:15 - INFO - finbert.utils -   input_ids: 101 2178 1772


Validation losses: [0.9659676849842072, 0.7577664852142334, 0.5364635288715363, 0.4611974507570267]
Epoch: 100%|██████████| 4/4 [11:37<00:00, 174.39s/it]


## Test the model

`bert.evaluate` outputs the DataFrame, where true labels and logit values for each example is given

In [44]:
test_data = finbert.get_data('test')

In [45]:
results = finbert.evaluate(examples=test_data, model=trained_model)

05/16/2020 13:55:26 - INFO - finbert.utils -   *** Example ***
05/16/2020 13:55:26 - INFO - finbert.utils -   guid: test-1
05/16/2020 13:55:26 - INFO - finbert.utils -   tokens: [CLS] in addition , cr ##amo and pea ##b have signed exclusive five - year rental agreements in finland and have extended their existing rental agreements in the swedish market for another five years . [SEP]
05/16/2020 13:55:26 - INFO - finbert.utils -   input_ids: 101 1999 2804 1010 13675 22591 1998 26034 2497 2031 2772 7262 2274 1011 2095 12635 10540 1999 6435 1998 2031 3668 2037 4493 12635 10540 1999 1996 4467 3006 2005 2178 2274 2086 1012 102 0 0 0 0 0 0 0 0 0 0 0 0
05/16/2020 13:55:26 - INFO - finbert.utils -   input_mask: 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0
05/16/2020 13:55:26 - INFO - finbert.utils -   segment_ids: 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
05/16/2020 13:55:26 - INFO - finbert

### Prepare the classification report

In [46]:
def report(df, cols=['label','prediction','logits']):
    #print('Validation loss:{0:.2f}'.format(metrics['best_validation_loss']))
    cs = CrossEntropyLoss(weight=finbert.class_weights)
    loss = cs(torch.tensor(list(df[cols[2]])),torch.tensor(list(df[cols[0]])))
    print("Loss:{0:.2f}".format(loss))
    print("Accuracy:{0:.2f}".format((df[cols[0]] == df[cols[1]]).sum() / df.shape[0]) )
    print("\nClassification Report:")
    print(classification_report(df[cols[0]], df[cols[1]]))

In [47]:
# results['prediction'] = results.predictions.apply(lambda x: np.argmax(x,axis=0))

In [48]:
# report(results,cols=['labels','prediction','predictions'])

### Get predictions

With the `predict` function, given a piece of text, we split it into a list of sentences and then predict sentiment for each sentence. The output is written into a dataframe. Predictions are represented in three different columns: 

1) `logit`: probabilities for each class

2) `prediction`: predicted label

3) `sentiment_score`: sentiment score calculated as: probability of positive - probability of negative

Below we analyze a paragraph taken out of [this](https://www.economist.com/finance-and-economics/2019/01/03/a-profit-warning-from-apple-jolts-markets) article from The Economist. For comparison purposes, we also put the sentiments predicted with TextBlob.
> Later that day Apple said it was revising down its earnings expectations in the fourth quarter of 2018, largely because of lower sales and signs of economic weakness in China. The news rapidly infected financial markets. Apple’s share price fell by around 7% in after-hours trading and the decline was extended to more than 10% when the market opened. The dollar fell by 3.7% against the yen in a matter of minutes after the announcement, before rapidly recovering some ground. Asian stockmarkets closed down on January 3rd and European ones opened lower. Yields on government bonds fell as investors fled to the traditional haven in a market storm.

In [49]:
text = "Later that day Apple said it was revising down its earnings expectations in \
the fourth quarter of 2018, largely because of lower sales and signs of economic weakness in China. \
The news rapidly infected financial markets. Apple’s share price fell by around 7% in after-hours \
trading and the decline was extended to more than 10% when the market opened. The dollar fell \
by 3.7% against the yen in a matter of minutes after the announcement, before rapidly recovering \
some ground. Asian stockmarkets closed down on January 3rd and European ones opened lower. \
Yields on government bonds fell as investors fled to the traditional haven in a market storm."

In [50]:
cl_path = project_dir/'models'/'classifier_model'/'finbert-sentiment'
model = BertForSequenceClassification.from_pretrained(cl_path, cache_dir=None, num_labels=3)

05/16/2020 13:55:33 - INFO - pytorch_pretrained_bert.modeling -   loading archive file /mnt/c/Users/Fazz/source/repos/finBERT/models/classifier_model/finbert-sentiment from cache at /mnt/c/Users/Fazz/source/repos/finBERT/models/classifier_model/finbert-sentiment
05/16/2020 13:55:33 - INFO - pytorch_pretrained_bert.modeling -   Model config {
  "attention_probs_dropout_prob": 0.1,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "max_position_embeddings": 512,
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "type_vocab_size": 2,
  "vocab_size": 30522
}



In [51]:
result = predict(text,model)

05/16/2020 13:55:40 - INFO - pytorch_pretrained_bert.tokenization -   loading vocabulary file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-vocab.txt from cache at /home/fazz/.pytorch_pretrained_bert/26bc1ad6c0ac742e9b52263248f6d0f00068293b33709fae12320c0e35ccfbbb.542ce4285a40d23a559526243235df47c5f75c197f04f37d1a0c124c32c9a084
05/16/2020 13:55:40 - INFO - finbert.utils -   *** Example ***
05/16/2020 13:55:40 - INFO - finbert.utils -   guid: 0
05/16/2020 13:55:40 - INFO - finbert.utils -   tokens: [CLS] later that day apple said it was rev ##ising down its earnings expectations in the fourth quarter of 2018 , largely because of lower sales and signs of economic weakness in china . [SEP]
05/16/2020 13:55:40 - INFO - finbert.utils -   input_ids: 101 2101 2008 2154 6207 2056 2009 2001 7065 9355 2091 2049 16565 10908 1999 1996 2959 4284 1997 2760 1010 4321 2138 1997 2896 4341 1998 5751 1997 3171 11251 1999 2859 1012 102 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 

In [52]:
blob = TextBlob(text)
result['textblob_prediction'] = [sentence.sentiment.polarity for sentence in blob.sentences]
result

,sentence,logit,prediction,sentiment_score,textblob_prediction
0,"Later that day Apple said it was revising down its earnings expectations in the fourth quarter of 2018, largely because of lower sales and signs of economic weakness in China.","[0.4157066, 0.41632047, 0.16797292]",negative,-0.000614,0.051746
1,The news rapidly infected financial markets.,"[0.28655145, 0.26793945, 0.44550914]",neutral,0.018612,0.000000
2,Apple’s share price fell by around 7% in after-hours trading and the decline was extended to more than 10% when the market opened.,"[0.7016184, 0.18024853, 0.11813314]",positive,0.521370,0.500000
3,"The dollar fell by 3.7% against the yen in a matter of minutes after the announcement, before rapidly recovering some ground.","[0.56493795, 0.3044215, 0.13064049]",positive,0.260516,0.000000
4,Asian stockmarkets closed down on January 3rd and European ones opened lower.,"[0.48739818, 0.36379233, 0.14880946]",positive,0.123606,-0.051111
5,Yields on government bonds fell as investors fled to the traditional haven in a market storm.,"[0.3402299, 0.28342327, 0.37634683]",neutral,0.056807,0.000000


In [53]:
print(f'Average sentiment is %.2f.' % (result.sentiment_score.mean()))

Average sentiment is 0.16.


Here is another example

In [54]:
text2 = "Shares in the spin-off of South African e-commerce group Naspers surged more than 25% \
in the first minutes of their market debut in Amsterdam on Wednesday. Bob van Dijk, CEO of \
Naspers and Prosus Group poses at Amsterdam's stock exchange, as Prosus begins trading on the \
Euronext stock exchange in Amsterdam, Netherlands, September 11, 2019. REUTERS/Piroschka van de Wouw \
Prosus comprises Naspers’ global empire of consumer internet assets, with the jewel in the crown a \
31% stake in Chinese tech titan Tencent. There is 'way more demand than is even available, so that’s \
good,' said the CEO of Euronext Amsterdam, Maurice van Tilburg. 'It’s going to be an interesting \
hour of trade after opening this morning.' Euronext had given an indicative price of 58.70 euros \
per share for Prosus, implying a market value of 95.3 billion euros ($105 billion). The shares \
jumped to 76 euros on opening and were trading at 75 euros at 0719 GMT."

In [55]:
result2 = predict(text2,model)
blob = TextBlob(text2)
result2['textblob_prediction'] = [sentence.sentiment.polarity for sentence in blob.sentences]

05/16/2020 13:55:46 - INFO - pytorch_pretrained_bert.tokenization -   loading vocabulary file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-vocab.txt from cache at /home/fazz/.pytorch_pretrained_bert/26bc1ad6c0ac742e9b52263248f6d0f00068293b33709fae12320c0e35ccfbbb.542ce4285a40d23a559526243235df47c5f75c197f04f37d1a0c124c32c9a084
05/16/2020 13:55:46 - INFO - finbert.utils -   *** Example ***
05/16/2020 13:55:46 - INFO - finbert.utils -   guid: 0
05/16/2020 13:55:46 - INFO - finbert.utils -   tokens: [CLS] shares in the spin - off of south african e - commerce group nas ##pers surged more than 25 % in the first minutes of their market debut in amsterdam on wednesday . [SEP]
05/16/2020 13:55:46 - INFO - finbert.utils -   input_ids: 101 6661 1999 1996 6714 1011 2125 1997 2148 3060 1041 1011 6236 2177 17235 7347 18852 2062 2084 2423 1003 1999 1996 2034 2781 1997 2037 3006 2834 1999 7598 2006 9317 1012 102 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
05/16

In [56]:
result2

,sentence,logit,prediction,sentiment_score,textblob_prediction
0,Shares in the spin-off of South African e-commerce group Naspers surged more than 25% in the first minutes of their market debut in Amsterdam on Wednesday.,"[0.5129372, 0.32087708, 0.16618575]",positive,0.192060,0.250000
1,"Bob van Dijk, CEO of Naspers and Prosus Group poses at Amsterdam's stock exchange, as Prosus begins trading on the Euronext stock exchange in Amsterdam, Netherlands, September 11, 2019.","[0.26633182, 0.14230451, 0.59136367]",neutral,0.124027,0.000000
2,"REUTERS/Piroschka van de Wouw Prosus comprises Naspers’ global empire of consumer internet assets, with the jewel in the crown a 31% stake in Chinese tech titan Tencent.","[0.20023952, 0.10992615, 0.6898343]",neutral,0.090313,0.000000
3,"There is 'way more demand than is even available, so that’s good,' said the CEO of Euronext Amsterdam, Maurice van Tilburg.","[0.41840225, 0.182257, 0.39934072]",positive,0.236145,0.533333
4,'It’s going to be an interesting hour of trade after opening this morning.',"[0.37664372, 0.2819541, 0.34140217]",positive,0.094690,0.500000
5,"Euronext had given an indicative price of 58.70 euros per share for Prosus, implying a market value of 95.3 billion euros ($105 billion).","[0.36853215, 0.109311715, 0.5221562]",neutral,0.259220,0.000000
6,The shares jumped to 76 euros on opening and were trading at 75 euros at 0719 GMT.,"[0.4040208, 0.09557458, 0.50040466]",neutral,0.308446,0.000000


In [57]:
print(f'Average sentiment is %.2f.' % (result2.sentiment_score.mean()))

Average sentiment is 0.19.
